In [7]:
import requests
import hashlib
from ProjectTree.ProjectTree import ProjectTree
import os

In [3]:
user = requests.get('https://staging2.osf.io:443/api/v2/users/?filter[family_name]=ojha').json()['data'][0] 
user

{'date_registered': '2015-02-03T15:38:29.024000',
 'educational_institutions': [],
 'employment_institutions': [],
 'family_name': 'Ojha',
 'fullname': 'Himanshu Ojha',
 'given_name': 'Himanshu',
 'gravatar_url': 'https://secure.gravatar.com/avatar/bbb26cc80350f8e4ae6cbff9c7fd1abd?d=identicon&size=40',
 'id': 'gpr3h',
 'links': {'html': 'https://staging2.osf.io/gpr3h/',
  'nodes': {'relation': 'https://staging2.osf.io/api/v2/users/gpr3h/nodes/'},
  'self': 'https://staging2.osf.io/api/v2/users/gpr3h/'},
 'middle_name': 'rock',
 'social_accounts': {},
 'suffix': '',
 'type': 'users'}

In [5]:
user_id = 'gpr3h'
projects = requests.get('https://staging2.osf.io:443/api/v2/users/{}/nodes/?filter[category]=project'.format(user_id))
myproject = projects.json()['data'][1]
myproject

{'category': 'project',
 'date_created': '2015-06-16T14:12:21.484000',
 'date_modified': '2015-06-17T21:03:33.557000',
 'description': 'polling the api for project and component changes',
 'id': 'zexc9',
 'links': {'children': {'count': 0,
   'related': 'https://staging2.osf.io/api/v2/nodes/zexc9/children/'},
  'contributors': {'count': 1,
   'related': 'https://staging2.osf.io/api/v2/nodes/zexc9/contributors/'},
  'files': {'related': 'https://staging2.osf.io/api/v2/nodes/zexc9/files/'},
  'html': 'https://staging2.osf.io/zexc9/',
  'pointers': {'count': 0,
   'related': 'https://staging2.osf.io/api/v2/nodes/zexc9/pointers/'},
  'registrations': {'count': 0,
   'related': 'https://staging2.osf.io/api/v2/nodes/zexc9/registrations/'},
  'self': 'https://staging2.osf.io/api/v2/nodes/zexc9/'},
 'properties': {'collection': False,
  'dashboard': False,
  'registration': False},
 'public': True,
 'tags': {'system': [], 'user': []},
 'title': 'poll',
 'type': 'nodes'}

In [ ]:
def check_file_changed(local_item, server_response):
    #generate md5 for server response file
    hasher = hashlib.md5()
    with open('/home/himanshu/OSF-Offline/sandbox/some_file.txt', 'rb') as afile:
        buf = afile.read()
        hasher.update(buf)
    local_hash = 
    print(hasher.hexdigest())

In [9]:
# files = requests.get(myproject['links']['files']['related']) #has both folders and files

local_project_folder = '/home/himanshu/OSF-Offline/sandbox/project/'

def check_file_folder(file_folder):
    if file_folder['item_type']=='folder':
        print('folder!')
        folder = requests.get(file_folder['links']['related']).json()['data']
        if not os.path.exists(file_folder['name']):
            os.makedirs('/home/himanshu/OSF-Offline/sandbox/osfstorage/{}'.format(file_folder['name']))
        for inner in folder:
            check_file_folder(inner)
    elif file_folder['item_type']=='file':
        print('file!')
        r = requests.get(file_folder['links']['self'])
        with open('/home/himanshu/OSF-Offline/sandbox/osfstorage/{}'.format(file_folder['name']), 'wb') as fd:
            for chunk in r.iter_content(2048): #todo: which is better? 1024 or 2048? Apparently, not much difference.
                fd.write(chunk)
            print('file SHOULD now be on local storage.')
        
        

def check_component(component):
    if not os.path.exists(os.path.join(local_project_folder, component['name'])):
            os.makedirs(os.path.join(local_project_folder, component['name']))
    files_folders = requests.get(component['links']['files']['related']).json()['data']
    for file_folder in files_folders:
        check_file_folder(file_folder)
    print(file_folder)
    
    child_components = []
    child_components_resp = requests.get(component['links']['children']['related']).json()
    child_components.extend(child_components_resp['data'])
    while child_components_resp['links']['meta']['next'] != None:
        child_components_resp = requests.get(component['links']['next']).json()
        child_components.extend(child_components_resp['data'])    
    for child_component in child_components:
        check_component(child_component)
        
        
check_component(myproject)

folder!
file!


file SHOULD now be on local storage.


file!
file SHOULD now be on local storage.


file!
file SHOULD now be on local storage.


file!
file SHOULD now be on local storage.

...


file!
file SHOULD now be on local storage.


folder!
file!


file SHOULD now be on local storage.


folder!


FileExistsError: [Errno 17] File exists: '/home/himanshu/OSF-Offline/sandbox/osfstorage/list of classes to take~'

In [15]:
hasher = hashlib.md5()
with open('/home/himanshu/OSF-Offline/sandbox/some_file.txt', 'rb') as afile:
    buf = afile.read()
    hasher.update(buf)
print(hasher.hexdigest())

d41d8cd98f00b204e9800998ecf8427e


In [14]:
r = requests.get('https://files-api-staging2.osf.io/file?path=%2F55802f0f404f772ee954abac&nid=zexc9&provider=osfstorage&cookie=5581dfd6404f774825676c14.5gyk4BEgYgZ0GDrlMsAdZ6MCJbY')
with open('/home/himanshu/OSF-Offline/sandbox/some_file.txt', 'wb') as fd:
    for chunk in r.iter_content(2048):
        fd.write(chunk)
    print('done!')

In [ ]:
def download_file(file_link, download_to):
    r = requests.get(file_link)
    with open(download_to, 'wb') as fd:
        for chunk in r.iter_content(2048): #todo: which is better? 1024 or 2048? Apparently, not much difference for client.
            fd.write(chunk)
        print('file SHOULD now be on local storage.')

def download_folder(file_folder, download_to):
    if file_folder['item_type']=='folder':
        print('folder!')
        folder = requests.get(file_folder['links']['related']).json()['data']
        if not os.path.exists(file_folder['name']):
            os.makedirs(os.path.join(download_to, file_folder['name']))
        for inner in folder:
            download_folder(inner, os.path.join(download_to,inner['name']))
    elif file_folder['item_type'] == 'file':
        print('file!')
        download_file(file_folder['links']['self'], os.path.join(download_to, file_folder['name']))

In [1]:
#this is the officialish final cell
import requests
import hashlib
from ProjectTree.ProjectTree import ProjectTree
import os
user = requests.get('https://staging2.osf.io:443/api/v2/users/?filter[family_name]=ojha').json()['data'][0] 
local_folder = '/home/himanshu/OSF-Offline/sandbox/OSF_FOLDER/'

def check_osf(user, local_folder):
    user_id = user['id']
    projects = requests.get('https://staging2.osf.io:443/api/v2/users/{}/nodes/?filter[category]=project'.format(user_id))
    projects = projects.json()['data']
    for project in projects:
        check_project(project,local_folder)

def check_project(project,local_folder):
    # if not os.path.exists(os.path.join(local_folder,project['name'])):
    #     os.makedirs(os.path.join(local_folder, project['name']))
    #todo: stuff to do here before checking project
    check_component(project, local_folder)
    
def check_file_folder(file_folder,local_folder):
    if file_folder['item_type']=='folder':
        print('folder!')
        new_local_folder = os.path.join(local_folder, file_folder['name'])
        folder = requests.get(file_folder['links']['related']).json()['data']
        if not os.path.exists(new_local_folder):
            os.makedirs(new_local_folder)
        for inner in folder:
            check_file_folder(inner,new_local_folder)
    elif file_folder['item_type']=='file':
        print('file!')
        new_local_file_path = os.path.join(local_folder,file_folder['name'])
        if not os.path.exists(new_local_file_path):
            r = requests.get(file_folder['links']['self'])
            with open(new_local_file_path, 'wb') as fd:
                for chunk in r.iter_content(2048): #todo: which is better? 1024 or 2048? Apparently, not much difference.
                    fd.write(chunk)
                print('file SHOULD now be on local storage.')
        
def check_component(component, local_folder):
    print(component)
    new_local_folder = os.path.join(local_folder, component['title'])
    if not os.path.exists(new_local_folder):
            os.makedirs(new_local_folder)
    files_folders = requests.get(component['links']['files']['related']).json()['data']
    for file_folder in files_folders:
        check_file_folder(file_folder,new_local_folder)
    
    child_components = []
    child_components_resp = requests.get(component['links']['children']['related']).json()
    print('---------------------------HIMANSHU----------------------------')
    print(child_components_resp)
    child_components.extend(child_components_resp['data'])
    while child_components_resp['links']['next'] != None:
        child_components_resp = requests.get(component['links']['next']).json()
        child_components.extend(child_components_resp['data'])    
    for child_component in child_components:
        check_component(child_component, new_local_folder)

check_osf(user, local_folder)

{'date_created': '2015-02-03T15:36:49.901000', 'type': 'nodes', 'links': {'registrations': {'count': 0, 'related': 'https://staging2.osf.io/api/v2/nodes/c8dkn/registrations/'}, 'contributors': {'count': 229, 'related': 'https://staging2.osf.io/api/v2/nodes/c8dkn/contributors/'}, 'pointers': {'count': 0, 'related': 'https://staging2.osf.io/api/v2/nodes/c8dkn/pointers/'}, 'self': 'https://staging2.osf.io/api/v2/nodes/c8dkn/', 'files': {'related': 'https://staging2.osf.io/api/v2/nodes/c8dkn/files/'}, 'children': {'count': 0, 'related': 'https://staging2.osf.io/api/v2/nodes/c8dkn/children/'}, 'html': 'https://staging2.osf.io/c8dkn/'}, 'category': 'project', 'id': 'c8dkn', 'title': 'Inverse optimal moderator', 'date_modified': '2015-02-28T23:04:40.832000', 'public': True, 'description': 'implement global experiences', 'tags': {'user': [], 'system': []}, 'properties': {'registration': False, 'collection': False, 'dashboard': False}}
folder!

TypeError: list indices must be integers, not str